# Synthetic Data Generation Baseline

In [1]:
import os, sys
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import json
import pickle

from sklearn.model_selection import train_test_split
sns.set_theme()
sys.path.append(os.path.abspath('..'))

In [2]:
%load_ext autoreload
%autoreload 2

from lib.vae_models import VAE, CVAE
import lib.datasets as datasets
import lib.utils as utils

#### Import Data

In [3]:
DATASET_NAME = 'goi4_dp_small_2yrs_inpost'
RANDOM_SEED = 2112
np.random.seed(RANDOM_SEED)

In [4]:
df = pd.read_csv(f'../data/{DATASET_NAME}/dataset.csv')
data, dates, users = df.iloc[:,:-2].values, df.date.values, df.user.values
date_ids, user_ids = df.date.unique(), df.user.unique()
num_days, num_users = len(date_ids), len(user_ids)
print(f'Loaded {len(data)} consumption profiles from {num_days} dates and {num_users} users')

Loaded 1011780 consumption profiles from 730 dates and 1386 users


In [5]:
date_dict = np.load(f'../data/{DATASET_NAME}/encode_dict.npy', allow_pickle=True).item()["date_dict"]
date_dict_inv = {v: k for k, v in date_dict.items()}

In [6]:
if not os.path.exists(f'../data/{DATASET_NAME}/raw_dates.npy'):
    raw_dates = np.array([datetime.datetime.strptime(date_dict_inv[d], '%Y-%m-%d') for d in dates])
    np.save(f'../data/{DATASET_NAME}/raw_dates.npy', raw_dates)
else:
    raw_dates = np.load(f'../data/{DATASET_NAME}/raw_dates.npy', allow_pickle=True)

### Prepare Conditions

In [7]:
months = np.array([d.month for d in raw_dates])
weekdays = np.array([d.weekday() for d in raw_dates])

In [8]:
condition_kwargs = {}

In [9]:
ADD_MONTHS = True
ADD_WEEKDAYS = True

condition_kwargs["tags"], condition_kwargs["types"], condition_kwargs["supports"]  = [], [], []
if ADD_MONTHS: 
    condition_kwargs["tags"].append("months")
    condition_kwargs["types"].append("circ")
    condition_kwargs["supports"].append(np.unique(months).tolist())
if ADD_WEEKDAYS: 
    condition_kwargs["tags"].append("weekdays")
    condition_kwargs["types"].append("circ")
    condition_kwargs["supports"].append(np.unique(weekdays).tolist())

In [10]:
conditioner = datasets.Conditioner(**condition_kwargs)

In [11]:
condition_set = conditioner.transform({"months": months, "weekdays": weekdays})

#### Set Resolution

In [12]:
RESOLUTION = 1 #in hours

if RESOLUTION == 12:
    X = np.reshape(data, (-1, 24))
    X = np.reshape(np.concatenate([X[:,6:], X[:,:6]], axis=-1), (num_users, num_days, int(24/RESOLUTION), int(RESOLUTION))).sum(axis=-1)    #circle shift the last dimension of X
else:
    X = np.reshape(data, (num_users, num_days, int(24/RESOLUTION), int(RESOLUTION))).sum(axis=-1)

condition_set = np.reshape(condition_set, (num_users, num_days, -1))

#### Clean Data

In [13]:
nonzero_user_mask = np.sum(np.all(X == 0, axis=2), axis=1) < num_days
print(f'Removing {(~nonzero_user_mask).sum()} users with all-zero consumption profiles')
positive_user_mask = np.sum(np.any(X < 0, axis=2), axis=1) == 0
print(f'Removing {(~positive_user_mask).sum()} users with any-negative consumption profiles')
user_mask = nonzero_user_mask & positive_user_mask
X = X[user_mask]
condition_set = condition_set[user_mask]

Removing 4 users with all-zero consumption profiles
Removing 18 users with any-negative consumption profiles


#### Ampute the Dataset

In [14]:
np.random.seed(RANDOM_SEED)
n, a, b = num_days, 0.85, 10.0
# n, a, b = num_days, 1.0, 1.0
missing_days = np.random.binomial(n, p=np.random.beta(a, b, size=X.shape[0]), size=X.shape[0])
print(f"Mean of missing days: {n*a/(a+b):.2f}")

Mean of missing days: 57.19


In [15]:
X_missing = X.copy().astype(float)
conditions_missing = condition_set.copy().astype(float)

for user in range(X.shape[0]): 
    X_missing[user, :missing_days[user]] = np.nan
    conditions_missing[user, :missing_days[user]] = np.nan

#### Subsample the Dataset

In [16]:
USER_SUBSAMPLE_RATE, DAY_SUBSAMPLE_RATE = 1, 1
X, X_missing = X[::USER_SUBSAMPLE_RATE, ::DAY_SUBSAMPLE_RATE, :], X_missing[::USER_SUBSAMPLE_RATE, ::DAY_SUBSAMPLE_RATE, :]
condition_set, conditions_missing = condition_set[::USER_SUBSAMPLE_RATE, ::DAY_SUBSAMPLE_RATE, :], conditions_missing[::USER_SUBSAMPLE_RATE, ::DAY_SUBSAMPLE_RATE, :]
num_users, num_days, num_features = X.shape
X_gt_list = [X[user, :missing_days[user]]*1 for user in range(num_users)]
X_gt_condition_list = [condition_set[user, :missing_days[user]]*1 for user in range(num_users)]

print("{:.<40}{:.>5}".format("Number of (subsampled/filtered) users", num_users))
print("{:.<40}{:.>5}".format("Number of (subsampled) days", num_days))
print("{:.<40}{:.>5}".format("Number of (aggregated) features", num_features))

Number of (subsampled/filtered) users....1364
Number of (subsampled) days...............730
Number of (aggregated) features............24


In [17]:
missing_idx_mat  = np.isnan(X_missing).any(2)
missing_num_labels = {"user": missing_idx_mat.sum(1), "day": missing_idx_mat.sum(0) }

In [18]:
X_missing = X_missing.reshape(-1, num_features)
conditions_missing = conditions_missing.reshape(-1, conditioner.cond_dim)
missing_idx = np.isnan(X_missing.sum(1))
X_missing = X_missing[~missing_idx]
conditions_missing = conditions_missing[~missing_idx]

#### Prepare the Training Data with Missing Records

In [19]:
nonzero_mean, nonzero_std = utils.zero_preserved_log_stats(X_missing)
X_missing = utils.zero_preserved_log_normalize(X_missing, nonzero_mean, nonzero_std)

In [20]:
## split the dataset using skitlearn
VAL_RATIO = 0.01

X_train, X_val, conditions_train, conditions_val = train_test_split(X_missing, conditions_missing, test_size=0.1, random_state=RANDOM_SEED)
X_train, X_val, conditions_train, conditions_val = torch.tensor(X_train).float(), torch.tensor(X_val).float(), torch.tensor(conditions_train).float(), torch.tensor(conditions_val).float()

In [21]:
trainset = datasets.ConditionedDataset(inputs=X_train, conditions=conditions_train)
valset = datasets.ConditionedDataset(inputs=X_val, conditions=conditions_val)
print(f"Number of Training Points: {len(trainset)}")
print(f"Number of Validation Points: {len(valset)}")

Number of Training Points: 825447
Number of Validation Points: 91717


### Model

In [22]:
model_kwargs = {
                "latent_dim": 10,
                "condition_dim": conditioner.cond_dim,
                "condencoding_dim": None,
                "posterior_dist": 'normal',
                "likelihood_dist": 'mixed',
                "learn_decoder_sigma": True,
                "num_neurons": 100,
                "num_hidden_layers": 3,
                "dropout": True,
                "dropout_rate": 0.25,
                "batch_normalization": True,
                }

In [23]:
model = CVAE(input_dim=num_features, **model_kwargs)

### Training

In [24]:
train_kwargs = {
                "lr": 1e-4,
                "beta": 1.0,
                "num_mc_samples": 1,
                "epochs": 1000,
                "verbose_freq": 100,
                "tensorboard": True,
                "batch_size": 1024,
                "validation_freq": 100
                }

In [25]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=train_kwargs["batch_size"], shuffle=True, drop_last=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=train_kwargs["batch_size"], drop_last=False)

In [26]:
model.fit(trainloader=trainloader, valloader=valloader, **train_kwargs)

Epoch:   0%|          | 0/1000 [00:00<?, ?it/s]/home/kbolat/miniconda3/envs/torchenv/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Epoch:   0%|          | 1/1000 [00:01<22:12,  1.33s/it]/home/kbolat/miniconda3/envs/torchenv/lib/python3.9/site-packages/torch/autograd/__init__.py:266: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
        

Validation -- ELBO=-5.94e+01 / RLL=-5.67e+01 / KL=2.64e+00
Iteration: 100 -- ELBO=-6.28e+01 / RLL=-5.99e+01 / KL=2.83e+00


KeyboardInterrupt: 

In [27]:
model.eval()

CVAE(
  (encoder): GaussianNN(
    (parameterizer): ParameterizerNN(
      (block_dict): ModuleDict(
        (input): NNBlock(
          (input_layer): Sequential(
            (0): Linear(in_features=28, out_features=50, bias=True)
            (1): Softplus(beta=1, threshold=20)
          )
          (middle_layers): ModuleList(
            (0-1): 2 x Sequential(
              (0): Linear(in_features=50, out_features=50, bias=True)
              (1): Softplus(beta=1, threshold=20)
            )
          )
          (output_layer): Sequential(
            (0): Linear(in_features=50, out_features=50, bias=True)
            (1): Softplus(beta=1, threshold=20)
            (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (3): Dropout(p=0.5, inplace=False)
          )
        )
        (mu): NNBlock(
          (input_layer): Sequential(
            (0): Linear(in_features=50, out_features=50, bias=True)
            (1): Softplus(beta=1, thresh

In [28]:
save_path = model.log_dir
model_name = f'trained_model'
model_path = f'./{save_path}/{model_name}.pt'
torch.save(model.state_dict(), model_path)
print(f'Model saved at {model_path}')

Model saved at ./runs/Apr22_16-52-52_iepg-st-gpu.ewi.tudelft.nl/trained_model.pt


In [35]:
conditioner_path = f'./{save_path}/conditioner.pkl'
with open(conditioner_path, 'wb') as f: pickle.dump(conditioner, f)
print(f'Conditioner saved at {conditioner_path}')

Conditioner saved at ./runs/Apr22_16-52-52_iepg-st-gpu.ewi.tudelft.nl/conditioner.pkl
